In [5]:
'''
### inputs
'''

Stocks_number = int(input("Enter the Number of Stocks You Want: "))

Stocks_name = []
for i in range(1, Stocks_number + 1):
    Stocks_name.append(input(f"Enter Stock's Name {i}: "))

av = int(input("Enter Number of Absolute Views: "))
rv = int(input("Enter Number of Relative Views: "))

while(True):
    if (rv + av > Stocks_number):
        print("Total Number of Views should be 3 or less")
        
        av = int(input("Enter Number of Absolute Views: "))
        rv = int(input("Enter Number of relative Views: "))
    else:
        break

absolute_name = []
relative_name = []
for i in range(1, av+1):
    absolute_name.append(input(f"Stock's Name for Absolute View {i}: "))

t = []
r = []
for i in range(1, rv+1):
    t = input(f"First Stock's Name for Relative View {i}: ")
    r = input(f"Second Stock's Name for Relative View {i}: ")
    relative_name.append([t , r])

print(absolute_name)
print(relative_name)

Ab_value = []
for i in range(0, av):
    Ab_value.append((float(input(f"Enter your Absolute View for stock {absolute_name[i]} in %: ")))/100)

R_value = []
for i in range(0, rv):
    R_value.append((float(input(f"Enter your Relative View for {relative_name[i]} in %: ")))/100)

Market_Caps = []
for i in range(0, Stocks_number):
    Market_Caps.append((float(input(f"Enter Market Cap of Stock {Stocks_name[i]} in: ")))/100)


'''
### imports
'''

import pandas as pd
import numpy as np
import math
from scipy.optimize import Bounds
from scipy.optimize import minimize
from numpy.linalg import inv
import requests

'''
### functions
'''

def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

# defining the sharpe ratio for calculating the optimal weights in optimization section
def sharpe(w):
    '''
    w is a N x 1 array
    mu is the Black-Litterman means and a 1 x N array
    sigma is the Black-Litterman covariance matrix and a N x N array
    '''
    sh = - (w.dot(mu))/math.sqrt((w.dot(sigma.dot(w.T))))
    return sh

# defining the constriant of sum of weights being equal to 1 for optimization section
def check_sum(w):
    return(sum(w) - 1)

'''
### Data
'''
##
## Constructing the string of data as an input for postman

b = '{'
for ix, val in enumerate(Stocks_name):
    b += "\"name\":\"{}\",\"upperband\":20,\"lowerband\":0".format(val)
    if ix == len(Stocks_name)-1:
        b += '}'
    else:
        b += '},{'

a = "{\"stocks\":["

c = "],\"risk_free\":25,\"start_date\":\"13960910\",\"end_date\":\"13990910\",\"period\":\"daily\",\"method\":\"scipy\",\"opt_func\":\"sharp\",\"industry_const\":{},\"linear_const\":[],\"opt_func_param\":{}}"

data_string = a+b+c

##
## Getting Data from postman

token = None

url = "http://192.168.33.13:8090/app/user/login/"

payload="{\n\t\"mobile\" : \"09396561733\",\n\t\"password\": \"12345678\"\n}"
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload).json()

token = response["data"]["token"]

url = "http://192.168.33.13:8090/app/optimization/optimizations?data=true"

payload= data_string

headers = {
  'Content-Type': 'application/json',
  'authorization': token
}

response = requests.request("POST", url, headers=headers, data=payload.encode('utf-8')).json()

##
## Coverting prices in dictionary to data frame

Market = response["data"]["market_date"].items()
df_Market = pd.DataFrame(Market, columns = ("Date", "Market_Price"))
df = df_Market

## Creating data frame of Stocks Prices

for i in range(0, Stocks_number):
    data = pd.DataFrame(response["data"]["stocks"][i]["data"].items(), columns = ("Date", f"{Stocks_name[i]}"))
    df = pd.merge(df, data, how = "left", on = "Date")

Data = df.ffill()

'''
### Computations
'''
##
## Defining the dataframe of daily returns, Market Cap weights and calculating the covariance matrix

# daily return T x N dataframe
Stocks_Data = Data.iloc[:, 2:Stocks_number+2]
daily_return = Stocks_Data.pct_change(1)

# Calculating weights based on Market caps
Cap_Weights = []
for item in Market_Caps:
    Cap_Weights.append(item/sum(Market_Caps))

# weights dataframe of 1 x N
w_eq = pd.DataFrame(Cap_Weights, index=Stocks_name, columns=["CapWeight"])

# covariance matrix, N x N dataframe
Sigma_prior = daily_return.cov()

# Compute Pi
pi = implied_returns(delta=2.5, sigma=Sigma_prior, w=w_eq)

# Computing matrices of q and p of views
q1 = []
for i in range(0, av):
    q1.append(Ab_value[i])
for i in range(0, rv):
    q1.append(R_value[i])
    
q = pd.Series(q1)
p = pd.DataFrame(0., index = Stocks_name, columns = range(0, av + rv)).T


# Dynamic views - creating matrix of views p
i = 0
j = 0
while(i < len(absolute_name)):
    while(j < len(absolute_name)):
        p.iloc[i][absolute_name[j]] = 1
        break
    j = j + 1
    i = i + 1

r = len(absolute_name)
c = r
t = len(relative_name)
l = 0
s = 0
while(l < r + t):
    while(s < t):
        p.iloc[c][relative_name[s][0]] = 1
        p.iloc[c][relative_name[s][1]] = -1
        break
    c = c + 1    
    s = s + 1
    l = l + 1


'''
### Optimization
'''

delta = 2.5
tau = 0.05 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_eq, Sigma_prior, p, q, tau = tau)

w = np.array(w_eq.values).T
mu = np.array([bl_mu]).T
sigma = np.array(bl_sigma)

cons = [{'type':'eq', 'fun': check_sum}]

# generating random weghits that sum to 1 for the initial guess
initial_guess = np.random.random(Stocks_number)
initial_guess /= initial_guess.sum()
x0 = initial_guess

Low_bound = []
High_bound = []
for i in range(0, Stocks_number):
    Low_bound.append(0)
    High_bound.append(1)

bounds = Bounds(Low_bound, High_bound )

res = minimize(sharpe, x0, method = 'SLSQP', constraints = cons, bounds = bounds)
res.x.round(2)


Enter the Number of Stocks You Want: 6
Enter Stock's Name 1: شلرد
Enter Stock's Name 2: بنیرو
Enter Stock's Name 3: شنفت
Enter Stock's Name 4: وغدیر
Enter Stock's Name 5: وبملت
Enter Stock's Name 6: حریل
Enter Number of Absolute Views: 3
Enter Number of Relative Views: 3
Stock's Name for Absolute View 1: شلرد
Stock's Name for Absolute View 2: بنیرو
Stock's Name for Absolute View 3: حریل
First Stock's Name for Relative View 1: شنفت
Second Stock's Name for Relative View 1: وغدیر
First Stock's Name for Relative View 2: وبملت
Second Stock's Name for Relative View 2: شلرد
First Stock's Name for Relative View 3: بنیرو
Second Stock's Name for Relative View 3: شنفت
['شلرد', 'بنیرو', 'حریل']
[['شنفت', 'وغدیر'], ['وبملت', 'شلرد'], ['بنیرو', 'شنفت']]
Enter your Absolute View for stock شلرد in %: 10
Enter your Absolute View for stock بنیرو in %: 5
Enter your Absolute View for stock حریل in %: 8
Enter your Relative View for ['شنفت', 'وغدیر'] in %: 5
Enter your Relative View for ['وبملت', 'شلرد'] in

array([0.25, 0.32, 0.  , 0.  , 0.14, 0.3 ])

In [6]:
p

,شلرد,بنیرو,شنفت,وغدیر,وبملت,حریل
0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,-1.0,0.0,0.0
4,-1.0,0.0,0.0,0.0,1.0,0.0
5,0.0,1.0,-1.0,0.0,0.0,0.0


In [7]:
q

0    0.10
1    0.05
2    0.08
3    0.05
4    0.02
5    0.08
dtype: float64